In [8]:
import os
import json
import glob
import random
import collections

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers



TYPES = ["FLAIR", "T1w", "T2w", "T1wCE"]
WHITE_THRESHOLD = 10 # out of 255
EXCLUDE = [109, 123, 709]

base_path = 'D:\zerobase\Brain_tumor'

train_df = pd.read_csv(base_path + "/train_df.csv")
test_df = pd.read_csv(base_path + '/test_df.csv')
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]
def load_dicom(path, size = 224):
    ''' 
    Reads a DICOM image, standardizes so that the pixel values are between 0 and 1, then rescales to 0 and 255
    
    Note super sure if this kind of scaling is appropriate, but everyone seems to do it. 
    '''
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in TYPES)
    
    patient_path = os.path.join(
        base_path + "/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]
IMAGE_SIZE = 128

def get_all_data_for_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', IMAGE_SIZE)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_for_test(image_type):
    global train_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', IMAGE_SIZE)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

def get_all_data_for_val(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', IMAGE_SIZE)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

X, y, trainidt = get_all_data_for_train('T1wCE')
X_test, testidt = get_all_data_for_test('T1wCE')
X.shape, y.shape, trainidt.shape

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

((12832, 128, 128), (12832,), (12832,))

In [3]:
X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = train_test_split(X, y, trainidt, test_size=0.2, random_state=40)


split = int(X.shape[0] * 0.8)

# X_train = X[:split]
# X_valid = X[split:]

# y_train = y[:split]
# y_valid = y[split:]

# trainidt_train = trainidt[:split]
# trainidt_valid = trainidt[split:]

X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, trainidt_train.shape, trainidt_valid.shape

(TensorShape([10265, 128, 128, 1]),
 (10265, 2),
 TensorShape([2567, 128, 128, 1]),
 (2567, 2),
 (10265,),
 (2567,))

In [4]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [5]:
np.random.seed(0)
random.seed(12)
tf.random.set_seed(12)

inpt = keras.Input(shape=(128, 128, 1))

h = keras.layers.experimental.preprocessing.Rescaling(1./255)(inpt)
# h = data_augmentation(h)

# convolutional layer!
h = keras.layers.Conv2D(32, kernel_size=(3, 3),strides=(1,1), activation="relu", name="Conv_1", padding="valid")(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(64, kernel_size=(3, 3),strides=(1,1), activation="relu", name="Conv_2", padding="same")(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# pooling layer
h = keras.layers.MaxPool2D(pool_size=(2,2))(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# convolutional layer!
h = keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", name="Conv_3",padding ="same")(h)
# h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# pooling layer
# h = keras.layers.MaxPool2D(pool_size=(1,1))(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", name="Conv_4",padding ="valid")(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", name="Conv_5",padding ="same")(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.MaxPool2D(pool_size=(2,2))(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Dropout(0.3)(h)   

h = keras.layers.Flatten()(h) 
# h = global_average_layer(h)
h = keras.layers.Dense(128, activation='relu')(h)   

output = keras.layers.Dense(2, activation="softmax")(h)

model = keras.Model(inpt, output)

from tensorflow.keras.optimizers import SGD
# opt = SGD(lr=0.1)

checkpoint_filepath = 'best_model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
save_weights_only=False,
monitor='val_auc',
mode='max',
save_best_only=True,
save_freq='epoch')

model.compile(loss='categorical_crossentropy',
             optimizer=tf.keras.optimizers.SGD(learning_rate =0.0001),
             metrics=[tf.keras.metrics.AUC()])

history = model.fit(x=X_train, y = y_train, epochs=50, callbacks=[model_checkpoint_callback], validation_data= (X_valid, y_valid))

Epoch 1/50
321/321 [==============================] - 49s 107ms/step - loss: 0.7614 - auc: 0.5847 - val_loss: 0.6931 - val_auc: 0.5739
Epoch 2/50
321/321 [==============================] - 32s 100ms/step - loss: 0.6706 - auc: 0.6504 - val_loss: 0.6758 - val_auc: 0.6165
Epoch 3/50
321/321 [==============================] - 32s 101ms/step - loss: 0.6400 - auc: 0.6874 - val_loss: 0.6544 - val_auc: 0.6593
Epoch 4/50
321/321 [==============================] - 32s 101ms/step - loss: 0.6112 - auc: 0.7218 - val_loss: 0.6348 - val_auc: 0.6902
Epoch 5/50
321/321 [==============================] - 32s 101ms/step - loss: 0.5942 - auc: 0.7432 - val_loss: 0.6247 - val_auc: 0.7052
Epoch 6/50
321/321 [==============================] - 32s 101ms/step - loss: 0.5712 - auc: 0.7700 - val_loss: 0.6176 - val_auc: 0.7150
Epoch 7/50
321/321 [==============================] - 32s 101ms/step - loss: 0.5581 - auc: 0.7846 - val_loss: 0.5998 - val_auc: 0.7399
Epoch 8/50
321/321 [==============================] - 3

In [47]:
def get_confusion_matrix(result3, threshold=0.5):

    confusion_matrix = [[0, 0], [0, 0]]

#     for i in range(len(result3)):
#         threshold = 1 if result3.loc[i, "MGMT_value_x"] > threshold else 0
#         confusion_matrix[result3.loc[i, "MGMT_value_y"]][threshold] += 1
        
    for idx, data in result3.iterrows():
        threshold = 1 if data.MGMT_value_x > threshold else 0
        
        print('data.MGMT_value_x : ',data.MGMT_value_x)
        print('threshold : ',threshold)
        print('data.MGMT_value_y : ',data.MGMT_value_y)
        
        confusion_matrix[int(data.MGMT_value_y)][threshold] += 1

    return confusion_matrix

def get_acc_recall(arr):
    acc = sum((arr[0][0], arr[1][1]))/sum((sum(arr[0]), sum(arr[1])))
    recall = arr[1][1] / sum(arr[1])
    print(f"Acc: {acc} \t Recall: {recall}")

In [23]:
def loss(y_true, y_pred):
    return - (1 - theta(y_true - margin) * theta(y_pred - margin)
                - theta(1 - margin - y_true) * theta(1 - margin - y_pred)
             ) * (y_true * K.log(y_pred + 1e-8) + (1 - y_true) * K.log(1 - y_pred + 1e-8))

import tensorflow as tf

margin = 0.6
theta = lambda t: (K.sign(t)+1.)/2.

def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x, inputs)
    return x


X_val, validt = get_all_data_for_val('T1wCE')

model_path = [
 './best_model.h5',
]

pred_list = []
for path in model_path:
    model_best = tf.keras.models.load_model(filepath=path,custom_objects={"loss":loss,'leaky_relu': tf.nn.leaky_relu,'mish':mish})
    
#     sample = pd.read_csv(base_path + '/test_df.csv')
    y_pred = model_best.predict(X_val)
    #pred = np.argmax(y_pred, axis=1)
    pred_list.append(y_pred)


sample = pd.read_csv(base_path + '/test_df.csv')

y_pred = sum(pred_list)/len(pred_list)

pred = np.argmax(y_pred, axis=1)

result=pd.DataFrame(testidt)
result[1]=pred 

result.columns=['BraTS21ID','MGMT_value']
result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample['BraTS21ID']

result2.to_csv('submission_best_score_2dcnn.csv',index=False)
result2
    

  0%|          | 0/117 [00:00<?, ?it/s]

,BraTS21ID,MGMT_value
0,107,0.409091
1,753,0.683333
2,303,0.650000
3,106,0.700000
4,171,0.863636
...,...,...
112,703,0.400000
113,21,0.187500
114,444,0.571429
115,95,0.468750


In [33]:
result2['MGMT_value_y'] = test_df['MGMT_value']
result2.columns=[['BrsTS21ID','MGMT_value_x','MGMT_value_y']]
result2

,BrsTS21ID,MGMT_value_x,MGMT_value_y
0,107,0.409091,1
1,753,0.683333,0
2,303,0.650000,1
3,106,0.700000,1
4,171,0.863636,1
...,...,...,...
112,703,0.400000,0
113,21,0.187500,0
114,444,0.571429,0
115,95,0.468750,0


In [49]:
arr = get_confusion_matrix(result2)
print(arr)
get_acc_recall(arr)

data.MGMT_value_x :  0.4090909090909091
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.6833333333333333
threshold :  1
data.MGMT_value_y :  0.0
data.MGMT_value_x :  0.65
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.7
threshold :  1
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.8636363636363636
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.6
threshold :  1
data.MGMT_value_y :  0.0
data.MGMT_value_x :  0.45454545454545453
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.7358490566037735
threshold :  1
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.45454545454545453
threshold :  0
data.MGMT_value_y :  0.0
data.MGMT_value_x :  0.625
threshold :  1
data.MGMT_value_y :  0.0
data.MGMT_value_x :  0.9090909090909091
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value_x :  0.8620689655172413
threshold :  1
data.MGMT_value_y :  0.0
data.MGMT_value_x :  0.6363636363636364
threshold :  0
data.MGMT_value_y :  1.0
data.MGMT_value